In [1]:
%pip install kagglehub

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:


from torch.utils.data import DataLoader

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader

import pandas as pd

In [3]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [4]:
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [5]:
class SimpleFCN(nn.Module):
    def __init__(self, input_size=87, hidden_size=16):
        super(SimpleFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 2)
        self.act = nn.ReLU()

    def forward(self, x):
        # x = self.fc0(x)
        return self.fc1(self.act(self.fc0(x)))

In [6]:
hyperparams = {
    "num_epochs": 64,
    "batch_size": 256,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.2}, # 1.0 -> no edges, 0.0 -> all edges
    "choose_thresholds_del": {"fc0": 0.1}, # 1.0 -> all edges, 0.0 -> no edges
    "threshold": 0.95,
    "min_delta_epoch_replace": 8,
    "window_size": 50000000000000000,
    "lr": 2e-5,
    "delete_after": 2,    
    "task_type": "classification",
    "fully_connected": False,
    "max_to_replace": 900 # None -> no limit
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 64, batch_size: 256, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.2}, choose_thresholds_del: {'fc0': 0.1}, threshold: 0.95, min_delta_epoch_replace: 8, window_size: 50000000000000000, lr: 2e-05, delete_after: 2, task_type: classification, fully_connected: False, max_to_replace: 900"

In [7]:
import kagglehub

dname = "dataset_phishing.csv"
path = kagglehub.dataset_download("shashwatwork/web-page-phishing-detection-dataset")

d:\Coding\PY\self-expanding-nets\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df_data = pd.read_csv(f'{path}\\{dname}')
print(df_data.shape) 
print(df_data.head())
print(df_data[df_data["status"] == "legitimate"].shape)
print(df_data[df_data["status"] == "phishing"].shape)
df_data.loc[df_data["status"] == "legitimate", "status"] = 1
df_data.loc[df_data["status"] == "phishing", "status"] = 0
X = df_data.iloc[:, 1:-1]
Y = df_data.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)



(11430, 89)
                                                 url  length_url  \
0              http://www.crestonwood.com/router.php          37   
1  http://shadetreetechnology.com/V4/validation/a...          77   
2  https://support-appleld.com.secureupdate.duila...         126   
3                                 http://rgipt.ac.in          18   
4  http://www.iracing.com/tracks/gateway-motorspo...          55   

   length_hostname  ip  nb_dots  nb_hyphens  nb_at  nb_qm  nb_and  nb_or  ...  \
0               19   0        3           0      0      0       0      0  ...   
1               23   1        1           0      0      0       0      0  ...   
2               50   1        4           1      0      1       2      0  ...   
3               11   0        2           0      0      0       0      0  ...   
4               15   0        2           2      0      0       0      0  ...   

   domain_in_title  domain_with_copyright  whois_registered_domain  \
0                0    

In [9]:
class TabularDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X.values, dtype=torch.float32)
        self.y = torch.tensor(y.astype(int).values, dtype=torch.long)  

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=hyperparams['batch_size'], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=hyperparams['batch_size'], shuffle=False)


In [10]:
model = SimpleFCN()
sparse_model = convert_dense_to_sparse_network(model, layers=[model.fc0], device=device)

In [11]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: fedornigretuk to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [12]:
wandb.finish()
run = wandb.init(
    project="senmodel-phishing",
    name=f"trash",
    config=hyperparams
)

In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=hyperparams['lr'], weight_decay=1e-4)
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, criterion, optimizer, hyperparams, device)

100%|██████████| 36/36 [00:00<00:00, 211.47it/s]


Epoch 1/64, Train Loss: 7077.6168, Val Loss: 7371.4661, Val Accuracy: 0.4790


100%|██████████| 36/36 [00:00<00:00, 205.40it/s]


Epoch 2/64, Train Loss: 7046.8763, Val Loss: 7371.4661, Val Accuracy: 0.4790


100%|██████████| 36/36 [00:00<00:00, 221.93it/s]


...

Epoch 62/64, Train Loss: 7072.1771, Val Loss: 7371.4661, Val Accuracy: 0.4790


100%|██████████| 36/36 [00:00<00:00, 201.02it/s]


Epoch 63/64, Train Loss: 7057.0996, Val Loss: 7371.4661, Val Accuracy: 0.4790


100%|██████████| 36/36 [00:00<00:00, 193.67it/s]


Epoch 64/64, Train Loss: 7060.4205, Val Loss: 7371.4661, Val Accuracy: 0.4790
